# Module 5: Prompt Augmentation & Generation

**Learning Objectives:**
- Structure effective RAG prompts
- Inject retrieved context optimally
- Implement citation systems

---

## Setup

First, let's import the libraries we'll need and set up our environment.

In [10]:
# Install required packages
from typing import List, Dict

print("✅ Setup complete!")

✅ Setup complete!


## Section 1: Understanding RAG Prompt Structure

In a basic RAG system, we need to:
1. **Retrieve** relevant chunks (covered in Module 4)
2. **Augment** the prompt with retrieved context (this module)
3. **Generate** a response using the LLM

### Components of a RAG Prompt

```
┌─────────────────────────────────────┐
│  SYSTEM MESSAGE                     │
│  - Define role and constraints      │
│  - Instruct to use only context     │
└─────────────────────────────────────┘
           ↓
┌─────────────────────────────────────┐
│  CONTEXT (Retrieved Chunks)         │
│  [Chunk 1] Source: doc1.pdf         │
│  Content: ...                       │
│                                     │
│  [Chunk 2] Source: doc2.pdf         │
│  Content: ...                       │
└─────────────────────────────────────┘
           ↓
┌─────────────────────────────────────┐
│  USER QUERY                         │
│  Question: {user_question}          │
└─────────────────────────────────────┘
           ↓
┌─────────────────────────────────────┐
│  INSTRUCTIONS                       │
│  - How to cite sources              │
│  - What to do if uncertain          │
└─────────────────────────────────────┘
```

### Basic RAG Prompt Template

In [ ]:
# Basic RAG prompt template
BASIC_RAG_TEMPLATE = """
You are a helpful assistant that answers questions based on provided context.
Use ONLY the information from the context below to answer the question.
If the answer cannot be found in the context, say "I don't have enough information to answer this question."

Context:
{context}

Question: {question}

Answer:
"""

# Example usage
sample_context = """
[1] RAG stands for Retrieval-Augmented Generation. It's a technique that combines information retrieval with text generation.
[2] The main benefit of RAG is that it allows LLMs to access external knowledge without retraining.
"""

sample_question = "What does RAG stand for?"

prompt = BASIC_RAG_TEMPLATE.format(
    context=sample_context,
    question=sample_question
)

print("Generated Prompt:")
print("="*50)
print(prompt)

### 🤔 Reflection

Why do we need explicit instructions like "use ONLY the information from the context"?

<details>
<summary>Click to see answer</summary>
Without explicit constraints, LLMs will use their pre-trained knowledge to answer questions, which defeats the purpose of RAG. We want answers grounded in our specific documents, not general knowledge that might be outdated or incorrect for our use case.
</details>

## Section 2: Context Injection Strategies

How we format and present retrieved chunks significantly impacts answer quality.

### Strategy 1: Simple Concatenation

In [ ]:
# Simulate retrieved chunks (normally from Module 4's retrieval system)
retrieved_chunks = [
    {
        "content": "Python is a high-level programming language known for its simple syntax and readability.",
        "source": "python_basics.pdf",
        "score": 0.89
    },
    {
        "content": "Python supports multiple programming paradigms including procedural, object-oriented, and functional programming.",
        "source": "python_guide.pdf",
        "score": 0.85
    },
    {
        "content": "Python was created by Guido van Rossum and first released in 1991.",
        "source": "python_history.pdf",
        "score": 0.72
    }
]

# Simple concatenation
def format_context_simple(chunks: List[Dict]) -> str:
    """Simply concatenate all chunk content."""
    return "\n\n".join([chunk["content"] for chunk in chunks])

context_simple = format_context_simple(retrieved_chunks)
print("Simple Concatenation:")
print("="*50)
print(context_simple)

### Strategy 2: Numbered Context (Recommended)

In [ ]:
def format_context_numbered(chunks: List[Dict]) -> str:
    """Format chunks with numbers for citation."""
    formatted = []
    for i, chunk in enumerate(chunks, 1):
        formatted.append(f"[{i}] {chunk['content']}")
    return "\n\n".join(formatted)

context_numbered = format_context_numbered(retrieved_chunks)
print("Numbered Context:")
print("="*50)
print(context_numbered)

### Strategy 3: Context with Metadata (Best for Production)

In [ ]:
def format_context_with_metadata(chunks: List[Dict]) -> str:
    """Format chunks with source metadata for better attribution."""
    formatted = []
    for i, chunk in enumerate(chunks, 1):
        formatted.append(
            f"[{i}] (Source: {chunk['source']}, Relevance: {chunk['score']:.2f})\n"
            f"{chunk['content']}"
        )
    return "\n\n".join(formatted)

context_with_metadata = format_context_with_metadata(retrieved_chunks)
print("Context with Metadata:")
print("="*50)
print(context_with_metadata)

### Comparison: Which is Better?

In [ ]:
print("Strategy Comparison:")
print("="*60)
print("\n1. Simple Concatenation")
print("   ✅ Pros: Clean, minimal tokens")
print("   ❌ Cons: No citations possible, hard to track sources")
print("\n2. Numbered Context")
print("   ✅ Pros: Enables citations, clear separation")
print("   ❌ Cons: Slightly more tokens")
print("   👍 Recommended: Good balance")
print("\n3. Context with Metadata")
print("   ✅ Pros: Full traceability, source attribution")
print("   ❌ Cons: Most tokens, may confuse some LLMs")
print("   👍 Best for: Production systems requiring full audit trail")

## Section 3: Prompt Templates for Different Use Cases

### Template 1: Basic Q&A

In [ ]:
QA_TEMPLATE = """
You are a helpful AI assistant. Answer the user's question based on the context provided below.

Important guidelines:
- Use ONLY information from the context
- If the answer is not in the context, say "I don't have enough information to answer this question."
- Be concise and accurate

Context:
{context}

Question: {question}

Answer:
"""

print(QA_TEMPLATE)

### Template 2: Q&A with Citations (Recommended)

In [ ]:
QA_WITH_CITATIONS_TEMPLATE = """
You are a helpful AI assistant. Answer the user's question based on the context provided below.

Important guidelines:
- Use ONLY information from the context
- Cite your sources using [1], [2], etc.
- If multiple sources support a claim, cite all: [1][2]
- If the answer is not in the context, say "I don't have enough information to answer this question."
- Be concise and accurate

Context:
{context}

Question: {question}

Answer (remember to cite sources):
"""

print(QA_WITH_CITATIONS_TEMPLATE)

### Template 3: Summarization

In [ ]:
SUMMARIZATION_TEMPLATE = """
You are a helpful AI assistant. Create a concise summary of the following information.

Guidelines:
- Include only the most important points
- Keep the summary under 150 words
- Use bullet points if appropriate
- Maintain factual accuracy

Content to summarize:
{context}

Summary:
"""

print(SUMMARIZATION_TEMPLATE)

### Template 4: Comparative Analysis

In [ ]:
COMPARISON_TEMPLATE = """
You are a helpful AI assistant. Compare and contrast the information provided in the context.

Guidelines:
- Identify similarities and differences
- Organize your comparison clearly
- Cite sources using [1], [2], etc.
- If information conflicts, present both sides
- Use ONLY information from the context

Context:
{context}

Question: {question}

Comparative Analysis:
"""

print(COMPARISON_TEMPLATE)

## Summary

### What We've Learned

1. **RAG Prompt Structure**
   - System message, context, query, instructions
   - Importance of explicit constraints

2. **Context Injection Strategies**
   - Simple concatenation vs numbered vs metadata
   - Best practices for formatting multiple chunks

3. **Prompt Templates**
   - Q&A, summarization, comparison templates
   - Citation-enforcing templates


### Key Takeaways

✅ Always be explicit about using only provided context

✅ Number chunks to enable citations

✅ Use low temperature (0.0-0.3) for factual RAG

✅ Test with diverse queries

✅ Keep implementations simple and focused

### Next Steps

In **Module 6**, we'll:
- Integrate everything from Modules 1-5
- Build a complete end-to-end RAG system

**You're now ready to generate responses from retrieved context!** 🚀